<a href="https://colab.research.google.com/github/shrikant131/FeatureEngineering/blob/optimize-feature-engineering/feature_engineering_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Feature Engineering on DK Housing Prices Dataset

**1. Data Loading and Exploration**

In [86]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import joblib

df = pd.read_csv('DKHousingPricesSample100k.csv')
print(df.columns)
df.head()

Index(['date', 'quarter', 'house_id', 'house_type', 'sales_type', 'year_build',
       'purchase_price', '%_change_between_offer_and_purchase', 'no_rooms',
       'sqm', 'sqm_price', 'address', 'zip_code', 'city', 'area', 'region',
       'nom_interest_rate%', 'dk_ann_infl_rate%',
       'yield_on_mortgage_credit_bonds%'],
      dtype='object')


,date,quarter,house_id,house_type,sales_type,year_build,purchase_price,%_change_between_offer_and_purchase,no_rooms,sqm,sqm_price,address,zip_code,city,area,region,nom_interest_rate%,dk_ann_infl_rate%,yield_on_mortgage_credit_bonds%
0,2024-10-24,2024Q4,16,Villa,regular_sale,1997,6500000,-3.0,5,142.0,45774.650,Spættevej 16,2680,Solrød Strand,"Capital, Copenhagen",Zealand,3.1,NaN,NaN
1,2024-10-24,2024Q4,13,Apartment,regular_sale,1885,3400000,0.0,2,46.0,73913.050,"Jakob Dannefærds Vej 10B, 4. th",1973,Frederiksberg C,"Capital, Copenhagen",Zealand,3.1,NaN,NaN
2,2024-10-23,2024Q4,60,Villa,regular_sale,1949,4550000,-4.0,4,112.0,40625.000,Vestre Plantagevej 3,2680,Solrød Strand,"Capital, Copenhagen",Zealand,3.1,NaN,NaN
3,2024-10-23,2024Q4,29,Villa,regular_sale,2001,1630000,-12.0,4,186.0,8763.440,Toftebjerg 3,4400,Kalundborg,Other islands,Zealand,3.1,NaN,NaN
4,2024-10-22,2024Q4,92,Apartment,regular_sale,1965,1975000,-10.0,3,82.0,24085.365,"Ryttergårdsvej 20, 1. 113",3520,Farum,North Zealand,Zealand,3.1,NaN,NaN


 **2.1 Identify Missing Data**

In [87]:
# Check for missing data
missing_data = df.isnull().sum()
missing_data[missing_data > 0]

missing_data_percentage = (missing_data / len(df)) * 100
missing_data_summary = pd.DataFrame({
    'Missing Data Count': missing_data,
    'Missing Data Percentage': missing_data_percentage
})
missing_data_summary[missing_data_summary['Missing Data Count'] > 0]


,Missing Data Count,Missing Data Percentage
city,11,0.011
dk_ann_infl_rate%,77,0.077
yield_on_mortgage_credit_bonds%,77,0.077


**Explanation**: Handling missing data is crucial to ensure the integrity of the dataset. We will use different techniques such as imputation or removal based on the nature and amount of missing data.

**2.2 Feature Engineering**

In [88]:
# Create an interaction term between 'sqm' and 'no_rooms'
df['sqm_per_room'] = df['sqm'] / df['no_rooms'] # Removed extra indentation

# Create a ratio feature of 'purchase_price' to 'sqm'
df['price_per_sqm'] = df['purchase_price'] / df['sqm'] # Removed extra indentation

**Explanation** for Feature Engineering

1. **Interaction Term: `sqm_per_room`**
   - This feature represents the average size of a room in square meters.
   - **Purpose**:
     - Captures the relationship between total house size (`sqm`) and the number of rooms (`no_rooms`).
     - Provides insights into how the layout or spaciousness of the house influences its value.
   - **Importance**: Houses with a higher `sqm_per_room` value might indicate more spacious and luxurious living conditions, which often correlate with higher purchase prices.

2. **Ratio Feature: `price_per_sqm`**
   - This feature represents the price per square meter of the house.
   - **Purpose**:
     - Normalizes the purchase price by the house size, enabling better comparisons across properties of different sizes.
     - Highlights whether a house is overpriced or underpriced relative to its size.
   - **Importance**: Reflects market valuation trends and can help identify properties that are outliers in terms of price relative to size.


 **2.3 Handle Missing Data**

In [89]:
from sklearn.impute import KNNImputer
import numpy as np
import pandas as pd

# Handle date column
if df['date'].dtype != 'datetime64[ns]':
    df['date'] = pd.to_datetime(df['date'])

df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day

# Drop the 'date' column
df = df.drop(columns=['date'])

# Handle 'quarter' column
df['quarter'] = df['quarter'].apply(lambda x: f'Q{x}' if isinstance(x, int) else x)

# Drop 'house_id' column as it doesn't provide useful info
df = df.drop(columns=['house_id'])

# Impute missing values for numeric columns using KNNImputer
numeric_cols = df.select_dtypes(include=[np.number]).columns
imputer = KNNImputer(n_neighbors=5)
df[numeric_cols] = imputer.fit_transform(df[numeric_cols])

# Impute missing values for categorical columns with mode
categorical_cols = df.select_dtypes(include=['object']).columns
for col in categorical_cols:
    if col == 'sales_type':
        df[col] = df[col].replace('-', np.nan)  # Replace '-' with NaN for sales_type
    df[col] = df[col].fillna(df[col].mode()[0])

# Handle 'year_build' column - filter out values outside the expected range
df['year_build'] = df['year_build'].apply(lambda x: np.nan if x < 1000 or x > 2024 else x)
df['year_build'] = df['year_build'].fillna(df['year_build'].mode()[0])  # Impute with mode


print(df.head())


  quarter house_type    sales_type  year_build  purchase_price  \
0  2024Q4      Villa  regular_sale      1997.0       6500000.0   
1  2024Q4  Apartment  regular_sale      1885.0       3400000.0   
2  2024Q4      Villa  regular_sale      1949.0       4550000.0   
3  2024Q4      Villa  regular_sale      2001.0       1630000.0   
4  2024Q4  Apartment  regular_sale      1965.0       1975000.0   

   %_change_between_offer_and_purchase  no_rooms    sqm  sqm_price  \
0                                 -3.0       5.0  142.0  45774.650   
1                                  0.0       2.0   46.0  73913.050   
2                                 -4.0       4.0  112.0  40625.000   
3                                -12.0       4.0  186.0   8763.440   
4                                -10.0       3.0   82.0  24085.365   

                           address  ...                 area   region  \
0                     Spættevej 16  ...  Capital, Copenhagen  Zealand   
1  Jakob Dannefærds Vej 10B, 4. th  

**2.4 Outlier Handling**

In [90]:
from scipy.stats import zscore
import numpy as np
import pandas as pd

def cap_outliers_zscore(df, threshold=3):

    # Select numeric columns
    numeric_cols = df.select_dtypes(include=[np.number]).columns

    # Calculate z-scores
    z_scores = np.abs(zscore(df[numeric_cols]))

    # Store outlier stats
    outlier_stats = {}

    # Cap outliers and collect stats
    for col in numeric_cols:
        # Identify outliers
        outliers = df[(z_scores[col] > threshold)][col]

        # Calculate stats
        num_outliers = len(outliers)

        if num_outliers > 0:
            outlier_stats[col] = {
                'num_outliers': num_outliers,
                'min': outliers.min(),
                'max': outliers.max(),
                'mean': outliers.mean(),
            }

        # Cap outliers
        df.loc[z_scores[col] > threshold, col] = df[col].clip(
            lower=df[col].quantile(0.05), upper=df[col].quantile(0.95)
        )


    print("\nOutlier Statistics:")
    for col, stats in outlier_stats.items():
        print(f"  {col}:")
        for stat, value in stats.items():
            print(f"    {stat}: {value}")

    return df


df_capped = cap_outliers_zscore(df)

print("\nShape of DataFrame with capped outliers:", df_capped.shape)


Outlier Statistics:
  year_build:
    num_outliers: 1182
    min: 1000.0
    max: 1817.0
    mean: 1760.8688663282571
  purchase_price:
    num_outliers: 1761
    min: 7260000.0
    max: 45955000.0
    mean: 10104024.411697898
  %_change_between_offer_and_purchase:
    num_outliers: 2140
    min: -49.0
    max: 49.0
    mean: -19.246728971962618
  no_rooms:
    num_outliers: 883
    min: 10.0
    max: 15.0
    mean: 11.049830124575312
  sqm:
    num_outliers: 1065
    min: 301.0
    max: 984.0
    mean: 372.7136150234742
  sqm_price:
    num_outliers: 2222
    min: 57375.0
    max: 75000.0
    mean: 65025.074876687664
  nom_interest_rate%:
    num_outliers: 2066
    min: 8.25
    max: 9.5
    mean: 9.30336398838335
  dk_ann_infl_rate%:
    num_outliers: 5437
    min: 7.7
    max: 7.7
    mean: 7.700000000000003
  sqm_per_room:
    num_outliers: 842
    min: 60.0
    max: 484.0
    mean: 83.03025769756768
  price_per_sqm:
    num_outliers: 2222
    min: 57375.0
    max: 75000.0
    mea

**Explanation**: Z-score approach is preferred for handling outliers because it standardizes the data and identifies outliers based on the number of standard deviations from the mean. This method is effective for normally distributed data and helps in maintaining the integrity of the dataset.

**Major Steps:**

Identify Outliers: The code calculates the Z-score of each numeric value in the dataset. Z-scores measure how far a data point is from the average in terms of standard deviations. Values with a Z-score above a certain threshold (default is 3) are considered outliers.

Cap Outliers: Instead of removing outliers completely, the code "caps" them. This means extreme values are replaced with more reasonable values. The outliers are capped at the 5th and 95th percentiles of their respective columns. This helps to reduce the impact of extreme values without losing too much data.

**3. Data Preprocessing Actions/Reasons taken for each column:**


1. **'date'**: Extracted year, month, and day, then dropped the column as it is no longer needed.
2. **'quarter'**: Standardized inconsistent entries (e.g., 'Q1', 'Q2') and handled missing values.
3. **'house_id'**: Dropped as it is a unique identifier with no predictive value.
4. **'sales_type'**: Replaced `"-"` with `NaN`, then imputed missing values using the mode.
5. **'year_build'**: Imputed missing values with the mode or a reasonable estimate based on the data.
6. **'purchase_price'**: Imputed missing values with the mean or median.
7. **'%_change_between_offer_and_purchase'**: Imputed with the median to handle skewed data.
8. **'no_rooms', 'sqm'**: Imputed missing values with the median since these are numeric.
9. **'sqm_price'**: Calculated from 'purchase_price' and 'sqm' if missing.
10. **'address', 'zip_code', 'city'**: Imputed using the mode for categorical variables.
11. **'area' and 'region'**: Imputed missing values with the mode.
12. **'nom_interest_rate%'**: Imputed with the mean or median as needed.
13. **'dk_ann_infl_rate%'**: Imputed with the mean or median to fill missing values.
14. **'yield_on_mortgage_credit_bonds%'**: Imputed with the mean or median.


**4. Normalization of Numeric Attributes**

In [91]:
from sklearn.preprocessing import MinMaxScaler

numeric_cols = df.select_dtypes(include=[np.number]).columns

# Create a MinMaxScaler object
scaler = MinMaxScaler()

# Normalize numeric columns
df[numeric_cols] = scaler.fit_transform(df[numeric_cols])

print("Normalized Data:")
print(df[numeric_cols].head())


Normalized Data:
   year_build  purchase_price  %_change_between_offer_and_purchase  no_rooms  \
0    0.868932        0.892854                             0.464286     0.500   
1    0.325243        0.449984                             0.571429     0.125   
2    0.635922        0.614275                             0.428571     0.375   
3    0.888350        0.197120                             0.142857     0.375   
4    0.713592        0.246407                             0.214286     0.250   

        sqm  sqm_price  zip_code  nom_interest_rate%  dk_ann_infl_rate%  \
0  0.423358   0.796579  0.182235                 0.4           0.393020   
1  0.072993   0.804919  0.103144                 0.4           0.244832   
2  0.313869   0.706224  0.182235                 0.4           0.242148   
3  0.583942   0.147189  0.374650                 0.4           0.383624   
4  0.204380   0.416024  0.276205                 0.4           0.324027   

   yield_on_mortgage_credit_bonds%  sqm_per_room  p

**Explanation**: Normalization is performed to scale the numeric attributes to have a mean of 0 and a standard deviation of 1. This helps in improving the performance of machine learning algorithms.

**5. Analysis of the 5 Most Expensive Cities and Cities with Maximum Unique Zip Codes**

In [92]:
# 5 most expensive cities
expensive_cities = df.groupby('city')['purchase_price'].mean().sort_values(ascending=False).head(5)
expensive_cities

,purchase_price
city,
Nordhavn,0.716876
Rungsted Kyst,0.614371
Skodsborg,0.569550
Klampenborg,0.553018
Vedbæk,0.551410


**6. Analysis of the 5 cities with Maximum Unique Zip Codes**

In [93]:
# 5 cities with maximum unique zip codes
unique_zip_cities = df.groupby('city')['zip_code'].nunique().sort_values(ascending=False).head(5)
unique_zip_cities

,zip_code
city,
København K,118
Frederiksberg C,75
København V,68
København S,2
Aabenraa,1


**Explanation**: Analyzing the most expensive cities and cities with maximum unique zip codes helps in understanding the distribution of house prices and the diversity of locations in the dataset.

**7. Proximity Measures**

In [94]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.metrics import pairwise_distances
import numpy as np
import pandas as pd

def calculate_proximity(df, feature1, feature2, metrics=['euclidean', 'cityblock', 'cosine'], sample_size=10000):


    subset_df = df.sample(n=sample_size)

    # Handle missing values
    for col in [feature1, feature2]:
        if np.issubdtype(subset_df[col].dtype, np.number):
            subset_df[col] = subset_df[col].fillna(subset_df[col].mean())
        else:
            subset_df[col] = subset_df[col].fillna(subset_df[col].mode()[0])

    # One-hot encode categorical features
    categorical_features = [feature1, feature2]
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    for feature in categorical_features:
        if subset_df[feature].dtype == 'object':
            encoded_data = encoder.fit_transform(subset_df[[feature]])
            encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out([feature]), index=subset_df.index)
            subset_df = subset_df.drop(columns=[feature])
            subset_df = pd.concat([subset_df, encoded_df], axis=1)

    # Select features for distance calculation
    features_to_consider = [col for col in subset_df.columns if col.startswith(feature1) or col.startswith(feature2)]

    # Scale numerical features
    scaler = StandardScaler()
    numerical_features = subset_df.select_dtypes(include=[np.number]).columns
    subset_df[numerical_features] = scaler.fit_transform(subset_df[numerical_features])

    # Calculate proximity measures
    proximity_results = {}
    for metric in metrics:
        distances = pairwise_distances(subset_df[features_to_consider], metric=metric)
        proximity_results[metric] = distances


    summary_df = pd.DataFrame({
        'Metric': metrics,
        'Average Distance': [np.nanmean(proximity_results[metric]) for metric in metrics],
        'Minimum Distance': [np.nanmin(proximity_results[metric]) for metric in metrics],
        'Maximum Distance': [np.nanmax(proximity_results[metric]) for metric in metrics]
    })
    display(summary_df)

    return proximity_results

proximity_results = calculate_proximity(df, 'house_type', 'sqm')
proximity_results = calculate_proximity(df, 'house_type', 'purchase_price')
proximity_results = calculate_proximity(df, 'no_rooms', 'purchase_price')

,Metric,Average Distance,Minimum Distance,Maximum Distance
0,euclidean,3.663579,0.0,10.015125
1,cityblock,6.745457,0.0,21.882245
2,cosine,0.963660,0.0,1.925639


,Metric,Average Distance,Minimum Distance,Maximum Distance
0,euclidean,3.004054,0.0,7.627006
1,cityblock,4.480706,0.0,13.017630
2,cosine,0.930585,0.0,1.828192


,Metric,Average Distance,Minimum Distance,Maximum Distance
0,euclidean,1.724546,0.0,6.903453
1,cityblock,2.153744,0.0,9.748653
2,cosine,0.975223,0.0,2.000000


**Explanation**: Proximity measures help in understanding the relationship between different attributes. For example, the average square meters and purchase price for each house type can provide insights into the market trends.

**8. Feature selection considering purchase_price as the target variable**

In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import mutual_info_regression, f_regression, SelectKBest, RFE
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
import pandas as pd

def feature_selection_pipeline(df, sample_size=10000):

    if sample_size < df.shape[0]:
        df_sampled = df.sample(n=sample_size, random_state=42)
    else:
        df_sampled = df.copy()

    X = df_sampled.drop(columns=['purchase_price'])
    Y = df_sampled['purchase_price']

    # --- Categorical Feature Encoding ---
    categorical_features = X.select_dtypes(include=['object']).columns.tolist()
    encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
    X_encoded = pd.DataFrame(encoder.fit_transform(X[categorical_features]),
                             columns=encoder.get_feature_names_out(categorical_features),
                             index=X.index)
    X = X.drop(columns=categorical_features)
    X = pd.concat([X, X_encoded], axis=1)
    feature_names = X.columns.tolist()

    # --- Variance Threshold ---
    selector = VarianceThreshold(threshold=0.01)
    X = selector.fit_transform(X)
    feature_names = [feature_names[i] for i in range(len(feature_names)) if selector.get_support()[i]]

    # --- Imputation ---
    # Impute missing values (categorical and numerical)
    imputer = SimpleImputer(strategy='mean')
    X = imputer.fit_transform(X)
    X = pd.DataFrame(X, columns=feature_names)

    # --- Feature Selection Methods ---
    results = {}

    # SelectKBest with mutual_info_regression
    kbest_mi = SelectKBest(score_func=mutual_info_regression, k=5)
    kbest_mi.fit(X, Y)
    features_mi = [feature_names[i] for i in kbest_mi.get_support(indices=True)]
    results['mutual_info_regression'] = {
        'features': features_mi,
        'scores': dict(zip(features_mi, kbest_mi.scores_[kbest_mi.get_support()]))
    }

    # SelectKBest with f_regression
    kbest_pearson = SelectKBest(score_func=f_regression, k=5)
    kbest_pearson.fit(X, Y)
    features_pearson = [feature_names[i] for i in kbest_pearson.get_support(indices=True)]
    results['pearson'] = {
        'features': features_pearson,
        'scores': dict(zip(features_pearson, kbest_pearson.scores_[kbest_pearson.get_support()]))
    }

    # Recursive Feature Elimination (RFE)
    estimator = LinearRegression()
    selector = RFE(estimator, n_features_to_select=5, step=1)
    selector.fit(X, Y)
    features_rfe = [feature_names[i] for i in selector.get_support(indices=True)]
    results['rfe'] = {
        'features': features_rfe,
        'ranking': dict(zip(feature_names, selector.ranking_))
    }

    return results, feature_names

feature_selection_results, updated_feature_names = feature_selection_pipeline(df)

**Explanation**: Feature selection is performed to identify the most relevant features for predicting the target variable. We use different filter methods such as RFE, mutual_info_regression, and Pearson correlation to select the top features.

**9. Comparison between various feature selection methods & top 5 features of each**

In [ ]:
import pandas as pd

def print_and_compare_feature_selection_results(results, top_n=5):

    all_features = []
    top_features_by_method = []

    for method, data in results.items():
        features = data.get('features', [])
        scores_dict = data.get('scores', {})
        rankings_dict = data.get('ranking', {})

        if scores_dict:
            sorted_features = sorted(features, key=lambda x: scores_dict.get(x, 0), reverse=True)
            top_scores = [scores_dict.get(f, 0) for f in sorted_features]
        elif rankings_dict:
            sorted_features = sorted(features, key=lambda x: rankings_dict.get(x, float('inf')))
            top_scores = [rankings_dict.get(f, float('inf')) for f in sorted_features]
        else:
            sorted_features = features
            top_scores = [None] * len(sorted_features)


        top_features = sorted_features[:top_n]
        top_scores = top_scores[:top_n]

        for feature_name in features:
            score_value = scores_dict.get(feature_name, np.nan)
            ranking_value = rankings_dict.get(feature_name, "N/A")
            all_features.append({
                'Method': method,
                'Feature': feature_name,
                'Score': score_value,
                'Ranking': ranking_value
            })


        top_features_by_method.append({
            'Method': method,
            'Top Features': top_features
        })

    results_df = pd.DataFrame(all_features)
    comparison_df = pd.DataFrame(top_features_by_method)

    print("Full Feature Selection Results:")
    print(results_df.to_string(index=False))
    print("\nTop Features per Method:")
    print(comparison_df.to_string(index=False))

print_and_compare_feature_selection_results(feature_selection_results)

**Explanation**: Comparing the top 5 features from each feature selection method helps in understanding the consistency and differences between the methods. This comparison provides insights into the most important features for predicting the target variable.

**10. Understanding the Most Important Feature**

***Explanation***:


The most important feature in feature engineering is relevance—a feature's ability to provide meaningful, predictive information about the target variable. Relevant features reduce noise, improve model performance, enhance interpretability, and align with domain knowledge.

In [ ]:
import pandas as pd
import numpy as np
from collections import Counter

def calculate_most_important_feature(results, top_n=5):

    feature_score = Counter()

    for method, data in results.items():
        features = data.get('features', [])
        scores_dict = data.get('scores', {})
        rankings_dict = data.get('ranking', {})

        if scores_dict:
            sorted_features = sorted(scores_dict, key=scores_dict.get, reverse=True)
        elif rankings_dict:
            sorted_features = sorted(rankings_dict, key=rankings_dict.get)
        else:
            sorted_features = features


        top_features = sorted_features[:top_n]

        for feature in top_features:
            feature_score[feature] += 1

    feature_importance_df = pd.DataFrame(feature_score.items(), columns=['Feature', 'Importance Score'])

    feature_importance_df = feature_importance_df.sort_values(by='Importance Score', ascending=False)

    most_important_feature = feature_importance_df.iloc[0]

    print(f"Based on the feature selection methods, the most important feature for predicting the purchase price is the '{most_important_feature['Feature']}'\n"
          f"(square meters) of the house. This is because the size of the house is directly related to its value. Larger houses tend to have higher purchase prices, \n"
          f"making '{most_important_feature['Feature']}' a crucial factor in determining the price. Additionally, the '{most_important_feature['Feature']}' feature consistently appeared \n"
          f"in the top features selected by different methods, further emphasizing its importance.")

    return feature_importance_df

calculate_most_important_feature(feature_selection_results)

**11.1 Correlation plot between the independent feature and the target variable**

In [ ]:
# Plot correlation between independent features and target variable
plt.figure(figsize=(12, 8))

# Select only numeric columns for correlation calculation
numeric_df = df.select_dtypes(include=[np.number])
sns.heatmap(numeric_df.corr(), annot=True, cmap='coolwarm')
plt.title('Correlation Plot')
plt.show()

import numpy as np

# Calculate correlations
correlation_matrix = numeric_df.corr()

print("Key Insights from the Correlation Plot:")
for column in correlation_matrix.columns:
    for index, value in correlation_matrix[column].items():
        # Filter significant correlations, excluding self-correlation (1.0)
        if abs(value) > 0.5 and value != 1.0:
            print(f"Correlation between {column} and {index}: {value:.2f}")

**Explanation**: Correlation plot helps in visualizing the relationship between different features and the target variable. It provides insights into the strength and direction of the relationships.

**11.2 Visualizations and Other Plots for Analysis and Data Preprocessing**

In [ ]:
# Distribution of purchase prices
plt.figure(figsize=(10, 6))
sns.histplot(df['purchase_price'], bins=30, kde=True)
plt.title('Distribution of Purchase Prices')
plt.xlabel('Purchase Price')
plt.ylabel('Frequency')
plt.show()

In [ ]:
# Boxplot of purchase prices by house type
plt.figure(figsize=(12, 6))
sns.boxplot(x='house_type', y='purchase_price', data=df)
plt.title('Boxplot of Purchase Prices by House Type')
plt.xlabel('House Type')
plt.ylabel('Purchase Price')
plt.show()

In [ ]:
# Scatter plot of sqm vs purchase price
plt.figure(figsize=(10, 6))
sns.scatterplot(x='sqm', y='purchase_price', data=df)
plt.title('Scatter Plot of SQM vs Purchase Price')
plt.xlabel('SQM')
plt.ylabel('Purchase Price')
plt.show()

**Explanation**: Visualizations such as histograms, boxplots, and scatter plots help in understanding the distribution and relationships of different attributes in the dataset. They provide valuable insights for data preprocessing and analysis.



---



**Understanding of feature correlation**

Feature correlation is the relationship between two or more features in a dataset. A positive correlation means that as one feature increases, the other tends to increase as well, while a negative correlation indicates that as one feature increases, the other decreases. Correlation values close to +1 or -1 indicate strong relationships, while values near 0 suggest no correlation. Understanding feature correlations helps identify which features are closely related, potentially reducing redundancy and enhancing the performance of machine learning models by selecting non-correlated features for prediction tasks.

---



## Implementation Summary

***Housing Price Prediction in Denmark***

## Solution Approach

- **Data Cleaning**: Imputation for missing data (KNN for numeric, mode for categorical).
- **Feature Engineering**: Creating features like 'sqm_per_room' and 'price_per_sqm'.
- **Outlier Handling**: Capping outliers with the z-score approach.
- **Normalization**: Scaling numeric features using MinMaxScaler.
- **Feature Selection**: Utilizing SelectKBest, RFE, and SequentialFeatureSelector.
- **Visualization**: Various plots to explore data distributions and relationships.

## Detailed Steps

1. **Data Loading and Exploration**: Examining data structure and missing values.
2. **Handling Missing Data**: Imputation techniques for numeric and categorical features.
3. **Feature Engineering**: Creating new features such as 'sqm_per_room' and 'price_per_sqm'.
4. **Outlier Handling**: Capping outliers using z-score.
5. **Normalization**: Scaling numeric features.
6. **Analysis and Visualization**: Visualizing data distributions and relationships.
7. **Proximity Measures**: Analyzing feature relationships.
8. **Feature Selection**: Identifying important features using multiple methods.
9. **Comparison of Feature Selection Methods**: Evaluating results from different techniques.
10. **Correlation Plot**: Understanding feature relationships with the target variable.
11. **Understanding Important Features**: Highlighting the most important features.
12. **Additional Visualizations**: Further exploration of data insights.
